In [1]:
import torch.nn.functional as F
import torch.optim as optim
import random
import math
import datetime
import csv
import re
import os
import getopt, sys
import pathlib
from pandas import ExcelWriter 

from random import shuffle
import itertools
from collections import defaultdict	
# from numpy import rank
import matplotlib.pyplot as plt
import seaborn as sns

import os
import pandas as pd
import numpy as np
import json
import math

import emoji
import demoji
demoji.download_codes()
import contractions
import re
import itertools
import regex

import io
import jsonlines
from datetime import datetime
from datetime import date # import datetime
from dateutil import parser
import matplotlib.pyplot as plt

from langdetect import detect
import nltk
nltk.download('words')
from nltk.corpus import words

import spacy
import urllib.request
import urllib.parse

from re import search

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from scipy.stats import ranksums
from tqdm import tqdm

import cython

import h5py

#from active_learning import ActiveLearningLabeler, RandomSampler, UncertaintySampler, KMeansUncertaintySampler, EntropySampler, SelfTrainingSampler

#from uncertainty_sampling import UncertaintySampling
#from pytorch_clusters import CosineClusters 
#from pytorch_clusters import Cluster

/tmp/ipykernel_589060/2501747831.py:28: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()
[nltk_data] Downloading package words to /home/aditi/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [287]:
# To remove the mentions of the female politicians based on keywords
import public_figure_dictionary

In [288]:
def replace_names(text):
    
    for key, value in public_figure_dictionary.REPLACEMENT_NAMES.items():
        #For normal mentions
        for key_item in enumerate(key):
            if key_item[1] in text:
                text = text.replace(key_item[1], value)
    
    return text

In [289]:
def remove_hash_mention(text):
    pattern  = re.compile(r'@([a-zA-Z0-9]{1,15})', re.I)
    if bool(pattern.search(text)) == True:
        words = text.split()
        new_text = ''
        for word in words:
            if word.startswith('#'):
                word=replace_names(word)
                word = re.sub('#', '', word)
                new_text += ' '+ word
            else:
                new_text += ' '+ word
            text = new_text.lstrip()
    else:
        pass
    return text

In [290]:
def remove_quoted(text):
    len_text = len(text.split())
    #print(len_text)
    # replace one type of quotes with other
    text = text.replace("\'", "\"")
    quoted = re.findall(r'"(.*?)"', text)
    if len(quoted) != 0:
        #print('Yes', quoted)
        count = 0
        for item in quoted:
            num = len(item.split())
            count += num
        if (len_text - count) < 7: # no intentions understood for unquoted words less than 7
            text = ''
        else:
            pass
    else:
        pass
    return text

In [291]:
def split_count(text):
    emoji_counter = 0
    digit_counter = 0
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.EMOJI_DATA for char in word): ## counting emojis in a text
            emoji_counter += 1
            # Remove from the given text the emojis
            text = text.replace(word, '') 
        if any(char.isdigit() for char in word): ### counting digits in a text
            digit_counter +=1
            text = text.replace(word, '')
    words_counter = len(text.split())

    return emoji_counter, words_counter, digit_counter

In [292]:
def check_digit(text):
    words = text.split()
    len_words = len(words)
    count = 0
    for word in words:
        if word.isnumeric():
            count+=1
        else:
            continue
    if count == len_words:
        res = True
    else:
        res = False
    return res

In [293]:
def check_transliteration(text):
    words = text.split()
    start_regex = re.compile('^[@_!#$%&*()<>?/\|}{~£€]')
    reg_ex = re.compile('[@_!#$%^&*()<>?/\|}{~]')
    end_regex = re.compile('\.+|!+$')
    new_text = ''
    if (reg_ex.search(text) == None): 
        pass
    else:
        for word in words:
            if not ((start_regex.search(word) == None) and (end_regex.search(word) == None)):
                new_text += ' ' + word
            else:
                # inspired from leet word dictionary in https://en.wikipedia.org/wiki/Leet
                word = re.sub('@|/-|\^', 'a', word)
                #word = re.sub('!', 'i', word)
                word = re.sub('β|!3|I3', 'b', word)
                #word = re.sub('ε|£|€', 'e', word)
                word = re.sub('\$', 's', word)
                word = re.sub('\#', 'h', word)
                word = re.sub('\+', 't', word)
                word = re.sub('gee', 'g', word)
                word = re.sub('I2', 'r', word)
                word = re.sub('&', 'and', word)
                word = re.sub('f\*ck|fck', 'fuck', word)
                word = re.sub('b\*tch|_itch', 'bitch', word)
                word = re.sub('sh\*t', 'shit', word)
                new_text += ' ' + word
            text = new_text.lstrip()
    return text

In [294]:
def clear_news(text):

    news_keywords = ['full interview', 'breaking news', '#politicslive', '#politics', '#politicsuk', 'news', '#FakeNews', '#fakenews', 'today\'s headline', '#news', 'today\'s news']
    if (text.startswith('LIVE: ') or text.startswith('HEADLINES: ') or any(k in text.lower() for k in news_keywords)):#(any(news_keywords) isin test.lower()):
        text = ''
    else:
        pass
    
    return text

In [295]:
def usual_preprocess(text):

    # remove extra spacing from text
    text = re.sub(' +', ' ', text)

    try:

        # regular expression for URLs
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        urls = url_pattern.findall(text)
        replacement_url = '[URL]'
        for url in urls:
            text = text.replace(url, replacement_url)

        # countering hashtags with words -- replaces any normal words containing a hashtag in the beginning with the actual word itself
        # e.g., #redhead --> redhead
        textList = text.split()
        x = ''
        for i in textList:
            if (i[0] == '#'):
                if (i[1:] in words.words()):
                    i = i.replace('#', '')
            x += ' '+ i
        text = x.lstrip()

        ## replacing some usual text emoticons to emojis before the next step
        ## this has to be done before the next step because the emojis would be counted then
        text = text.replace(r":)","🙂").replace(r":-)","🙂")
        text = text.replace(r":(","🙁").replace(r":-(","🙁")
        text = text.replace(r"xD", "😆").replace(r"XD", "😆").replace(r"xd", "😆")
        text = text.replace(r":'‑(", '😢').replace(":'(", "😢")
        text = text.replace(r":D", "😃").replace(":-D", "😃")
        text = text.replace(r";)", "😉").replace(";-)", "😉")

        ## replace some general text alterations which would otherwise go undetected
        text = text.replace(r"&amp;", "and")

        ## check if text has only emojis or is empty
        counter = split_count(text)
        if (counter[1] == 0) and (check_digit(text) == True):
            text = ''
        else:
            ## now demojize the emojis from the text
            text = emoji.demojize(text)

            ## expanding on contractions
            text = contractions.fix(text)

            ## remove email IDs if present
            text = re.sub("[\w]+@[\w]+\.[c][o][m]", "[EMAIL]", text)

            ## replacing usernames in the text with generic keyword '@USERNAME'
            #text=remove_username_mentions(text)

            ## removing news if present
            text = clear_news(text)

            ## removing quotation if present much more than actual context as it would reduce the inference capability
            text = remove_quoted(text)

            ## check for special characters in string 
            #### THIS STEP TO BE DONE BEFORE THE LAST STEP
            #text = check_transliteration(text)

            # removing unicode emojis from the text
            # at the end for removing any trailing emoji patterns which add no value to the text
            text = text.encode('ascii', 'ignore').decode('ascii')
    
    except IndexError:
        text = ''

    return text

In [296]:
def collect_hashtags(dict_list):
    try:
        if math.isnan(float(dict_list)):
            tags = np.nan
    except TypeError: # since it otherwise gives type error if the other condition is not fulfilled
        tag_list = []
        for d in dict_list:
            tag_list.append(d.get('tag'))
        tags = ', '.join(str(x) for x in tag_list)
    return tags

In [297]:
feb_8_14_path = 'data/orig/Feb/8-14/'
feb_22_28_path = 'data/orig/Feb/22-28/'
mar_path = 'data/orig/Mar/15-23/'
apr_15_22_path = 'data/orig/Apr/15-22/'
apr_23_30_path = 'data/orig/Apr/23-30/'
may_path = 'data/orig/May/8-14/'
sep_path = 'data/orig/Sep/1-7/'
oct_path = 'data/orig/Oct/24-31/'

In [298]:
interested_files = ['AngelaRayner.jsonl', 'CarolineLucas.jsonl', 'jessphillips.jsonl', 'pritipatel.jsonl', 'SuellaBraverman.jsonl', 'HackneyAbbott.jsonl', 'trussliz.jsonl']

In [299]:
# add the items into lists for each folder
jsonl_files = [pos_jsonl for pos_jsonl in os.listdir(feb_8_14_path) if pos_jsonl.endswith('.jsonl')]
jsonl_files += [pos_jsonl for pos_jsonl in os.listdir(feb_22_28_path) if pos_jsonl.endswith('.jsonl')]
jsonl_files += [pos_jsonl for pos_jsonl in os.listdir(mar_path) if pos_jsonl.endswith('.jsonl')]
jsonl_files += [pos_jsonl for pos_jsonl in os.listdir(apr_15_22_path) if pos_jsonl.endswith('.jsonl')]
jsonl_files += [pos_jsonl for pos_jsonl in os.listdir(apr_23_30_path) if pos_jsonl.endswith('.jsonl')]
jsonl_files += [pos_jsonl for pos_jsonl in os.listdir(may_path) if pos_jsonl.endswith('.jsonl')]
jsonl_files += [pos_jsonl for pos_jsonl in os.listdir(sep_path) if pos_jsonl.endswith('.jsonl')]
jsonl_files += [pos_jsonl for pos_jsonl in os.listdir(oct_path) if pos_jsonl.endswith('.jsonl')]

jsonl_files = list(set(jsonl_files))

jsonl_files

['trussliz.jsonl',
 'AngelaRayner.jsonl',
 'jessphillips.jsonl',
 'CarolineLucas.jsonl',
 'SuellaBraverman.jsonl',
 'HackneyAbbott.jsonl',
 'pritipatel.jsonl']

In [300]:
data = pd.DataFrame()

for lst in jsonl_files:
    if lst in interested_files:
        # for each folder, we read the files separately
        with open(f'{feb_8_14_path}/{lst}') as f:
            data_temp_3 = f.read().splitlines()
            data_temp_3 = pd.DataFrame(data_temp_3)
            data = pd.concat([data, data_temp_3], axis = 0)  
        with open(f'{feb_22_28_path}/{lst}') as f:
            data_temp_3 = f.read().splitlines()
            data_temp_3 = pd.DataFrame(data_temp_3)
            data = pd.concat([data, data_temp_3], axis = 0)  
        with open(f'{mar_path}/{lst}') as f:
            data_temp_3 = f.read().splitlines()
            data_temp_3 = pd.DataFrame(data_temp_3)
            data = pd.concat([data, data_temp_3], axis = 0)  
        with open(f'{apr_15_22_path}/{lst}') as f:
            data_temp_1 = f.read().splitlines()
            data_temp_1 = pd.DataFrame(data_temp_1)
            data = pd.concat([data, data_temp_1], axis = 0)  
        with open(f'{apr_23_30_path}/{lst}') as f:
            data_temp_2 = f.read().splitlines()
            data_temp_2 = pd.DataFrame(data_temp_2)
            data = pd.concat([data, data_temp_2], axis = 0)  
        with open(f'{may_path}/{lst}') as f:
            data_temp_3 = f.read().splitlines()
            data_temp_3 = pd.DataFrame(data_temp_3)
            data = pd.concat([data, data_temp_3], axis = 0)  
        with open(f'{sep_path}/{lst}') as f:
            data_temp_3 = f.read().splitlines()
            data_temp_3 = pd.DataFrame(data_temp_3)
            data = pd.concat([data, data_temp_3], axis = 0)  
        with open(f'{oct_path}/{lst}') as f:
            data_temp_3 = f.read().splitlines()
            data_temp_3 = pd.DataFrame(data_temp_3)
            data = pd.concat([data, data_temp_3], axis = 0)  
    else:
        continue


data = data.reset_index(drop=True)
data
  

,0
0,"{""data"": [{""id"": ""1493374491073949700"", ""text""..."
1,"{""data"": [{""possibly_sensitive"": false, ""autho..."
2,"{""data"": [{""id"": ""1493360061816745991"", ""entit..."
3,"{""data"": [{""edit_history_tweet_ids"": [""1493355..."
4,"{""data"": [{""in_reply_to_user_id"": ""25227444"", ..."
...,...
20410,"{""data"": [{""created_at"": ""2022-10-24T07:49:04...."
20411,"{""data"": [{""text"": ""@pritipatel Shameless Tory..."
20412,"{""data"": [{""text"": ""@alisonsheena1 @jaco45er @..."
20413,"{""data"": [{""entities"": {""mentions"": [{""start"":..."


In [301]:
data.columns = ['json_element']

data = pd.json_normalize(data['json_element'].apply(json.loads))
'''
All columns:
'data', 'errors', 'includes.users', 'includes.tweets', 'includes.media',
'meta.newest_id', 'meta.oldest_id', 'meta.result_count',
'meta.next_token', '__twarc.url', '__twarc.version',
'__twarc.retrieved_at', 'includes.places', 'includes.polls'
'''
# only choosing the data['data']
data = pd.json_normalize(data['data']) ## newer version for 2022
data = data.stack().reset_index(drop=True)
data

0          {'id': '1493374491073949700', 'text': '@trussl...
1          {'id': '1493374437403549707', 'text': '@trussl...
2          {'id': '1493374427651887106', 'text': '@danoha...
3          {'id': '1493374389236035584', 'text': '@HappyU...
4          {'id': '1493374370600951810', 'text': 'Eritrea...
                                 ...                        
1894399    {'author_id': '315264228', 'reply_settings': '...
1894400    {'author_id': '1570852626371317762', 'reply_se...
1894401    {'author_id': '218066634', 'reply_settings': '...
1894402    {'author_id': '3331174174', 'reply_settings': ...
1894403    {'author_id': '1141105535707688960', 'reply_se...
Length: 1894404, dtype: object

In [302]:
df = data.apply(pd.Series)
df

,id,text,context_annotations,reply_settings,edit_history_tweet_ids,referenced_tweets,lang,author_id,in_reply_to_user_id,created_at,...,entities.annotations,entities.hashtags,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,attachments.poll_ids,withheld.copyright,withheld.country_codes,entities.cashtags,withheld.scope
0,1493374491073949700,"@trussliz It’s all about you, isn’t it?","[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374491073949700],"[{'type': 'replied_to', 'id': '149323354004916...",en,1662054668,65357102,2022-02-14T23:59:58.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1493374437403549707,@trussliz Your credibility. Drain.\nYour party...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374437403549707],"[{'type': 'replied_to', 'id': '149323354004916...",en,1116096809624064000,65357102,2022-02-14T23:59:46.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1493374427651887106,@danohagan @trussliz @Conservatives Yep. So am...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374427651887106],"[{'type': 'replied_to', 'id': '149323549459085...",en,50587630,20538964,2022-02-14T23:59:43.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1493374389236035584,@HappyUlster @jameshhowe @13MoonWillow @benhab...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374389236035584],"[{'type': 'replied_to', 'id': '149335656017260...",en,1473025918864089101,1360729792086220800,2022-02-14T23:59:34.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1493374370600951810,Eritrean troops remain in Ethiopia despite Int...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374370600951810],NaN,en,1472224058502754309,NaN,2022-02-14T23:59:30.000Z,...,"[{'start': 26, 'end': 33, 'probability': 0.951...","[{'start': 196, 'end': 219, 'tag': 'WithdrawEr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1894399,1584339302133403648,Will be interesting to see how big the Conserv...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584339302133403648],NaN,en,315264228,NaN,2022-10-24T00:21:39.000Z,...,"[{'start': 39, 'end': 56, 'probability': 0.598...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1894400,1584337821782810625,@pritipatel I hope you run for prime minister ...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584337821782810625],"[{'type': 'replied_to', 'id': '158375551501574...",en,1570852626371317762,61660254,2022-10-24T00:15:46.000Z,...,"[{'start': 66, 'end': 67, 'probability': 0.970...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1894401,1584336964157083648,@pritipatel @BorisJohnson You've got to laugh ...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584336964157083648],"[{'type': 'replied_to', 'id': '158378181349936...",en,218066634,61660254,2022-10-24T00:12:22.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1894402,1584336301335384064,@pritipatel Pity he has seen the light not sta...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584336301335384064],"[{'type': 'replied_to', 'id': '158375551501574...",en,3331174174,61660254,2022-10-24T00:09:44.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [303]:
df = df.drop_duplicates(subset='text')
df = df[df['lang'] == 'en']
df = df.drop(columns='lang')
df = df.reset_index(drop=True)
df

,id,text,context_annotations,reply_settings,edit_history_tweet_ids,referenced_tweets,author_id,in_reply_to_user_id,created_at,possibly_sensitive,...,entities.annotations,entities.hashtags,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,attachments.poll_ids,withheld.copyright,withheld.country_codes,entities.cashtags,withheld.scope
0,1493374491073949700,"@trussliz It’s all about you, isn’t it?","[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374491073949700],"[{'type': 'replied_to', 'id': '149323354004916...",1662054668,65357102,2022-02-14T23:59:58.000Z,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1493374437403549707,@trussliz Your credibility. Drain.\nYour party...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374437403549707],"[{'type': 'replied_to', 'id': '149323354004916...",1116096809624064000,65357102,2022-02-14T23:59:46.000Z,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1493374427651887106,@danohagan @trussliz @Conservatives Yep. So am...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374427651887106],"[{'type': 'replied_to', 'id': '149323549459085...",50587630,20538964,2022-02-14T23:59:43.000Z,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1493374389236035584,@HappyUlster @jameshhowe @13MoonWillow @benhab...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374389236035584],"[{'type': 'replied_to', 'id': '149335656017260...",1473025918864089101,1360729792086220800,2022-02-14T23:59:34.000Z,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1493374370600951810,Eritrean troops remain in Ethiopia despite Int...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374370600951810],NaN,1472224058502754309,NaN,2022-02-14T23:59:30.000Z,False,...,"[{'start': 26, 'end': 33, 'probability': 0.951...","[{'start': 196, 'end': 219, 'tag': 'WithdrawEr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593610,1584341774461710338,@pritipatel The kiss of death minister Priti u...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584341774461710338],"[{'type': 'replied_to', 'id': '158375551501574...",919072807,61660254,2022-10-24T00:31:29.000Z,False,...,"[{'start': 39, 'end': 43, 'probability': 0.901...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1593611,1584337821782810625,@pritipatel I hope you run for prime minister ...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584337821782810625],"[{'type': 'replied_to', 'id': '158375551501574...",1570852626371317762,61660254,2022-10-24T00:15:46.000Z,False,...,"[{'start': 66, 'end': 67, 'probability': 0.970...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1593612,1584336964157083648,@pritipatel @BorisJohnson You've got to laugh ...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584336964157083648],"[{'type': 'replied_to', 'id': '158378181349936...",218066634,61660254,2022-10-24T00:12:22.000Z,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1593613,1584336301335384064,@pritipatel Pity he has seen the light not sta...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584336301335384064],"[{'type': 'replied_to', 'id': '158375551501574...",3331174174,61660254,2022-10-24T00:09:44.000Z,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [304]:
df = df[df['possibly_sensitive'] == False]
df = df.drop(columns='possibly_sensitive')
df

,id,text,context_annotations,reply_settings,edit_history_tweet_ids,referenced_tweets,author_id,in_reply_to_user_id,created_at,conversation_id,...,entities.annotations,entities.hashtags,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,attachments.poll_ids,withheld.copyright,withheld.country_codes,entities.cashtags,withheld.scope
0,1493374491073949700,"@trussliz It’s all about you, isn’t it?","[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374491073949700],"[{'type': 'replied_to', 'id': '149323354004916...",1662054668,65357102,2022-02-14T23:59:58.000Z,1493233540049162242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1493374437403549707,@trussliz Your credibility. Drain.\nYour party...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374437403549707],"[{'type': 'replied_to', 'id': '149323354004916...",1116096809624064000,65357102,2022-02-14T23:59:46.000Z,1493233540049162242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1493374427651887106,@danohagan @trussliz @Conservatives Yep. So am...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374427651887106],"[{'type': 'replied_to', 'id': '149323549459085...",50587630,20538964,2022-02-14T23:59:43.000Z,1493233540049162242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1493374389236035584,@HappyUlster @jameshhowe @13MoonWillow @benhab...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374389236035584],"[{'type': 'replied_to', 'id': '149335656017260...",1473025918864089101,1360729792086220800,2022-02-14T23:59:34.000Z,1492639410889842689,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1493374370600951810,Eritrean troops remain in Ethiopia despite Int...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1493374370600951810],NaN,1472224058502754309,NaN,2022-02-14T23:59:30.000Z,1493374370600951810,...,"[{'start': 26, 'end': 33, 'probability': 0.951...","[{'start': 196, 'end': 219, 'tag': 'WithdrawEr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593610,1584341774461710338,@pritipatel The kiss of death minister Priti u...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584341774461710338],"[{'type': 'replied_to', 'id': '158375551501574...",919072807,61660254,2022-10-24T00:31:29.000Z,1583755515015749633,...,"[{'start': 39, 'end': 43, 'probability': 0.901...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1593611,1584337821782810625,@pritipatel I hope you run for prime minister ...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584337821782810625],"[{'type': 'replied_to', 'id': '158375551501574...",1570852626371317762,61660254,2022-10-24T00:15:46.000Z,1583755515015749633,...,"[{'start': 66, 'end': 67, 'probability': 0.970...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1593612,1584336964157083648,@pritipatel @BorisJohnson You've got to laugh ...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584336964157083648],"[{'type': 'replied_to', 'id': '158378181349936...",218066634,61660254,2022-10-24T00:12:22.000Z,1583781813499367425,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1593613,1584336301335384064,@pritipatel Pity he has seen the light not sta...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",everyone,[1584336301335384064],"[{'type': 'replied_to', 'id': '158375551501574...",3331174174,61660254,2022-10-24T00:09:44.000Z,1583755515015749633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [305]:
df = df[df['reply_settings'] == 'everyone']
df = df.drop(columns='reply_settings')
df

,id,text,context_annotations,edit_history_tweet_ids,referenced_tweets,author_id,in_reply_to_user_id,created_at,conversation_id,public_metrics.retweet_count,...,entities.annotations,entities.hashtags,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,attachments.poll_ids,withheld.copyright,withheld.country_codes,entities.cashtags,withheld.scope
0,1493374491073949700,"@trussliz It’s all about you, isn’t it?","[{'domain': {'id': '10', 'name': 'Person', 'de...",[1493374491073949700],"[{'type': 'replied_to', 'id': '149323354004916...",1662054668,65357102,2022-02-14T23:59:58.000Z,1493233540049162242,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1493374437403549707,@trussliz Your credibility. Drain.\nYour party...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",[1493374437403549707],"[{'type': 'replied_to', 'id': '149323354004916...",1116096809624064000,65357102,2022-02-14T23:59:46.000Z,1493233540049162242,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1493374427651887106,@danohagan @trussliz @Conservatives Yep. So am...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",[1493374427651887106],"[{'type': 'replied_to', 'id': '149323549459085...",50587630,20538964,2022-02-14T23:59:43.000Z,1493233540049162242,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1493374389236035584,@HappyUlster @jameshhowe @13MoonWillow @benhab...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",[1493374389236035584],"[{'type': 'replied_to', 'id': '149335656017260...",1473025918864089101,1360729792086220800,2022-02-14T23:59:34.000Z,1492639410889842689,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1493374370600951810,Eritrean troops remain in Ethiopia despite Int...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",[1493374370600951810],NaN,1472224058502754309,NaN,2022-02-14T23:59:30.000Z,1493374370600951810,0,...,"[{'start': 26, 'end': 33, 'probability': 0.951...","[{'start': 196, 'end': 219, 'tag': 'WithdrawEr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593610,1584341774461710338,@pritipatel The kiss of death minister Priti u...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",[1584341774461710338],"[{'type': 'replied_to', 'id': '158375551501574...",919072807,61660254,2022-10-24T00:31:29.000Z,1583755515015749633,0,...,"[{'start': 39, 'end': 43, 'probability': 0.901...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1593611,1584337821782810625,@pritipatel I hope you run for prime minister ...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",[1584337821782810625],"[{'type': 'replied_to', 'id': '158375551501574...",1570852626371317762,61660254,2022-10-24T00:15:46.000Z,1583755515015749633,0,...,"[{'start': 66, 'end': 67, 'probability': 0.970...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1593612,1584336964157083648,@pritipatel @BorisJohnson You've got to laugh ...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",[1584336964157083648],"[{'type': 'replied_to', 'id': '158378181349936...",218066634,61660254,2022-10-24T00:12:22.000Z,1583781813499367425,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1593613,1584336301335384064,@pritipatel Pity he has seen the light not sta...,"[{'domain': {'id': '10', 'name': 'Person', 'de...",[1584336301335384064],"[{'type': 'replied_to', 'id': '158375551501574...",3331174174,61660254,2022-10-24T00:09:44.000Z,1583755515015749633,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [306]:
## for columns in withheld
cols = [c for c in df.columns if c.lower()[:8] != 'withheld']
df=df[cols]
## for columns in geo
cols = [c for c in df.columns if c.lower()[:3] != 'geo']
df=df[cols]
df = df[df.columns[~df.columns.str.endswith('id')]]
cols = [c for c in df.columns if c.lower()[:4] != 'edit']
df=df[cols]
cols = [c for c in df.columns if c.lower()[:10] != 'attachment']
df=df[cols]

In [307]:
df = df.explode('context_annotations')
df = df.explode('referenced_tweets')

df_json_A = pd.json_normalize(df['context_annotations'])
df_json_B = pd.json_normalize(df['referenced_tweets'])

df['index'] = df.index
df_json_A['index'] = df_json_A.index
df_json_B['index'] = df_json_B.index
df_json_B = df_json_B.rename(columns={"id": "reference_tweet_id"}, errors="raise")


df = df.merge(df_json_A, how='inner', on='index')
df = df.merge(df_json_B, how='inner', on='index')

df = df.drop(columns=['context_annotations', 'referenced_tweets'])

#df = df.explode('edit_history_tweet_ids')

df['created_at'] = df['created_at'].map(lambda x: parser.isoparse(x).strftime('%Y-%m-%d'))
# convert to datetime format
df['created_at'] = df['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())

df = df.explode('entities.mentions')

df = df.explode('entities.urls')

df = df.explode('entities.annotations')

df = df.drop(columns=['entities.mentions', 'entities.urls', 'entities.annotations'])

df = df.reset_index(drop=True)

df

,text,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,entities.hashtags,entities.cashtags,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,type,reference_tweet_id
0,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
1,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
2,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
3,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
4,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83005484,Priti Patel would not let them into the country,2022-10-24,0,0,1,0,0,NaN,NaN,1593614,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,quoted,1516010386981539843
83005485,Priti Patel would not let them into the country,2022-10-24,0,0,1,0,0,NaN,NaN,1593614,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,quoted,1516010386981539843
83005486,Priti Patel would not let them into the country,2022-10-24,0,0,1,0,0,NaN,NaN,1593614,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,quoted,1516010386981539843
83005487,Priti Patel would not let them into the country,2022-10-24,0,0,1,0,0,NaN,NaN,1593614,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,quoted,1516010386981539843


In [308]:
df.shape

(83005489, 18)

In [309]:
df

,text,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,entities.hashtags,entities.cashtags,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,type,reference_tweet_id
0,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
1,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
2,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
3,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
4,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83005484,Priti Patel would not let them into the country,2022-10-24,0,0,1,0,0,NaN,NaN,1593614,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,quoted,1516010386981539843
83005485,Priti Patel would not let them into the country,2022-10-24,0,0,1,0,0,NaN,NaN,1593614,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,quoted,1516010386981539843
83005486,Priti Patel would not let them into the country,2022-10-24,0,0,1,0,0,NaN,NaN,1593614,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,quoted,1516010386981539843
83005487,Priti Patel would not let them into the country,2022-10-24,0,0,1,0,0,NaN,NaN,1593614,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,quoted,1516010386981539843


In [310]:
df = df.drop_duplicates(subset='text')
df = df.reset_index(drop=True)
df

,text,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,entities.hashtags,entities.cashtags,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,type,reference_tweet_id
0,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
1,@trussliz Your credibility. Drain.\nYour party...,2022-02-14,0,0,0,0,0,NaN,NaN,1,35,Politician,"Politicians in the world, like Joe Biden",857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
2,@danohagan @trussliz @Conservatives Yep. So am...,2022-02-14,0,0,0,0,0,NaN,NaN,2,131,Unified Twitter Taxonomy,A taxonomy of user interests.,847878884917886977,Politics,Politics,replied_to,1493233540049162242
3,@HappyUlster @jameshhowe @13MoonWillow @benhab...,2022-02-14,0,0,0,0,0,NaN,NaN,3,131,Unified Twitter Taxonomy,A taxonomy of user interests.,857212714300489729,Liz Truss,Liz Truss,replied_to,1493233540049162242
4,Eritrean troops remain in Ethiopia despite Int...,2022-02-14,0,0,0,0,0,"[{'start': 196, 'end': 219, 'tag': 'WithdrawEr...",NaN,4,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1070032753834438656,Political figures,Politician,replied_to,1493233540049162242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576509,@pritipatel The kiss of death minister Priti u...,2022-10-24,0,0,0,0,0,NaN,NaN,1593610,164,Fields of study,"for all fields of study, e.g., futurology, mic...",1278093262406815744,Fields of study,NaN,quoted,1516010386981539843
1576510,@pritipatel I hope you run for prime minister ...,2022-10-24,0,0,0,0,0,NaN,NaN,1593611,164,Fields of study,"for all fields of study, e.g., futurology, mic...",1278093262406815744,Fields of study,NaN,replied_to,1514631058234425344
1576511,@pritipatel @BorisJohnson You've got to laugh ...,2022-10-24,0,0,0,0,0,NaN,NaN,1593612,10,Person,Named people in the world like Nelson Mandela,10040395078,Joe Biden,US President Joe Biden,quoted,1516010386981539843
1576512,@pritipatel Pity he has seen the light not sta...,2022-10-24,0,0,0,0,0,NaN,NaN,1593613,10,Person,Named people in the world like Nelson Mandela,10040395078,Joe Biden,US President Joe Biden,replied_to,1514631058234425344


In [311]:
df = df[df['type'] == 'replied_to']
df = df.drop(columns='type')
df = df.reset_index(drop=True)
df

,text,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,entities.hashtags,entities.cashtags,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,reference_tweet_id
0,"@trussliz It’s all about you, isn’t it?",2022-02-14,0,0,0,0,0,NaN,NaN,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1493233540049162242
1,@trussliz Your credibility. Drain.\nYour party...,2022-02-14,0,0,0,0,0,NaN,NaN,1,35,Politician,"Politicians in the world, like Joe Biden",857212714300489729,Liz Truss,Liz Truss,1493233540049162242
2,@danohagan @trussliz @Conservatives Yep. So am...,2022-02-14,0,0,0,0,0,NaN,NaN,2,131,Unified Twitter Taxonomy,A taxonomy of user interests.,847878884917886977,Politics,Politics,1493233540049162242
3,@HappyUlster @jameshhowe @13MoonWillow @benhab...,2022-02-14,0,0,0,0,0,NaN,NaN,3,131,Unified Twitter Taxonomy,A taxonomy of user interests.,857212714300489729,Liz Truss,Liz Truss,1493233540049162242
4,Eritrean troops remain in Ethiopia despite Int...,2022-02-14,0,0,0,0,0,"[{'start': 196, 'end': 219, 'tag': 'WithdrawEr...",NaN,4,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1070032753834438656,Political figures,Politician,1493233540049162242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137980,"@pritipatel That's a joke, right?",2022-10-24,0,0,0,0,0,NaN,NaN,1593605,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1100765157339234304,Sky News,Sky News,1514631058234425344
1137981,@pritipatel @BorisJohnson How did that go then...,2022-10-24,0,0,0,0,0,NaN,NaN,1593607,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1196447117297905665,News outlets,NaN,1514631058234425344
1137982,@pritipatel Thanks Priti for staying loyal and...,2022-10-24,0,0,0,0,0,NaN,NaN,1593609,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1278093262406815744,Fields of study,NaN,1514631058234425344
1137983,@pritipatel I hope you run for prime minister ...,2022-10-24,0,0,0,0,0,NaN,NaN,1593611,164,Fields of study,"for all fields of study, e.g., futurology, mic...",1278093262406815744,Fields of study,NaN,1514631058234425344


In [312]:
df['hashtags'] = df['entities.hashtags'].map(lambda x: collect_hashtags(x))
print('Completed hashtags, moving onto cashtags ------>')

df['cashtags'] = df['entities.cashtags'].map(lambda x: collect_hashtags(x))
print('Completed cashtags too')

df = df.drop(columns=['entities.hashtags', 'entities.cashtags'])

Completed hashtags, moving onto cashtags ------>
Completed cashtags too


In [313]:
df = df.rename(columns={'public_metrics.like_count': 'like_count', 'public_metrics.retweet_count': 'retweet_count',
                        'public_metrics.quote_count': 'quote_count', 'public_metrics.retweet_count': 'retweet_count',
                        'public_metrics.reply_count': 'reply_count'})

In [314]:
df = df.drop(columns=['public_metrics.impression_count'])

In [315]:
df['text'] = df['text'].apply(lambda x: usual_preprocess(x))

In [316]:
#df_24_26 = df_24_26.drop(columns='type')
df = df.drop_duplicates(subset='text')
df = df.dropna(subset=['text'])
df = df.reset_index(drop=True)
df

,text,created_at,retweet_count,reply_count,like_count,quote_count,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,reference_tweet_id,hashtags,cashtags
0,"@trussliz It is all about you, is not it?",2022-02-14,0,0,0,0,0,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1493233540049162242,NaN,NaN
1,@trussliz Your credibility. Drain. Your party....,2022-02-14,0,0,0,0,1,35,Politician,"Politicians in the world, like Joe Biden",857212714300489729,Liz Truss,Liz Truss,1493233540049162242,NaN,NaN
2,@danohagan @trussliz @Conservatives Yep. So am...,2022-02-14,0,0,0,0,2,131,Unified Twitter Taxonomy,A taxonomy of user interests.,847878884917886977,Politics,Politics,1493233540049162242,NaN,NaN
3,@HappyUlster @jameshhowe @13MoonWillow @benhab...,2022-02-14,0,0,0,0,3,131,Unified Twitter Taxonomy,A taxonomy of user interests.,857212714300489729,Liz Truss,Liz Truss,1493233540049162242,NaN,NaN
4,Eritrean troops remain in Ethiopia despite Int...,2022-02-14,0,0,0,0,4,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1070032753834438656,Political figures,Politician,1493233540049162242,WithdrawEritreanTroops,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029824,"@pritipatel That is a joke, right?",2022-10-24,0,0,0,0,1593605,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1100765157339234304,Sky News,Sky News,1514631058234425344,NaN,NaN
1029825,@pritipatel @BorisJohnson How did that go then...,2022-10-24,0,0,0,0,1593607,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1196447117297905665,News outlets,NaN,1514631058234425344,NaN,NaN
1029826,@pritipatel Thanks Priti for staying loyal and...,2022-10-24,0,0,0,0,1593609,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1278093262406815744,Fields of study,NaN,1514631058234425344,NaN,NaN
1029827,@pritipatel I hope you run for prime minister ...,2022-10-24,0,0,0,0,1593611,164,Fields of study,"for all fields of study, e.g., futurology, mic...",1278093262406815744,Fields of study,NaN,1514631058234425344,NaN,NaN


In [317]:
df['public_metrics'] = df[['retweet_count', 'reply_count', 'like_count', 'quote_count']].sum(axis=1)
df = df.sort_values(by='public_metrics', ascending=False)

df

,text,created_at,retweet_count,reply_count,like_count,quote_count,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,reference_tweet_id,hashtags,cashtags,public_metrics
345107,Yes @trussliz absolutely smashed it babe!!!,2022-09-05,7747,2611,145129,675,557681,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1196447117297905665,News outlets,NaN,1497534323616096260,NaN,NaN,156162
822052,The rise of Suella Braverman is one of the gen...,2022-10-25,9759,1373,56143,473,1274643,131,Unified Twitter Taxonomy,A taxonomy of user interests.,847878884917886977,Politics,Politics,1505193990773260289,NaN,NaN,67748
277038,"General Election please! 81,000 Tory members v...",2022-09-06,5299,5231,54180,661,419155,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1498344952790671360,NaN,NaN,65371
819760,Appointing Suella Braverman as Home secretary ...,2022-10-25,12061,2073,50089,604,1271722,123,Ongoing News Story,Ongoing News Stories like 'Brexit',1484601166080081920,Russo-Ukrainian conflict,NaN,1505285549820682245,NaN,NaN,64827
726789,Disgusted to hear Suella Braverman say there i...,2022-10-31,9959,5332,48714,601,1138495,131,Unified Twitter Taxonomy,A taxonomy of user interests.,857212714300489729,Liz Truss,Liz Truss,1504117989150543874,NaN,NaN,64606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029805,@wegojojo @pritipatel @BorisJohnson How do you...,2022-10-24,0,0,0,0,1593575,35,Politician,"Politicians in the world, like Joe Biden",857212714300489729,Liz Truss,Liz Truss,1515603714194776067,NaN,NaN,0
1029807,@pritipatel I am guessing you are on board for...,2022-10-24,0,0,0,0,1593577,131,Unified Twitter Taxonomy,A taxonomy of user interests.,857212714300489729,Liz Truss,Liz Truss,1515603714194776067,GeneralElectionNow,NaN,0
1029808,@RishiSunak And gave @pritipatel a job who had...,2022-10-24,0,0,0,0,1593578,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1070032753834438656,Political figures,Politician,1515603714194776067,NaN,NaN,0
1029809,@pritipatel Mentally challenged turd,2022-10-24,0,0,0,0,1593579,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1515603714194776067,NaN,NaN,0


In [318]:
df = df[df['public_metrics'] > 5] # only considering the tweets that had some form of engagement. As a lot of data, we chose the threshold as 5
df

,text,created_at,retweet_count,reply_count,like_count,quote_count,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,reference_tweet_id,hashtags,cashtags,public_metrics
345107,Yes @trussliz absolutely smashed it babe!!!,2022-09-05,7747,2611,145129,675,557681,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1196447117297905665,News outlets,NaN,1497534323616096260,NaN,NaN,156162
822052,The rise of Suella Braverman is one of the gen...,2022-10-25,9759,1373,56143,473,1274643,131,Unified Twitter Taxonomy,A taxonomy of user interests.,847878884917886977,Politics,Politics,1505193990773260289,NaN,NaN,67748
277038,"General Election please! 81,000 Tory members v...",2022-09-06,5299,5231,54180,661,419155,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1498344952790671360,NaN,NaN,65371
819760,Appointing Suella Braverman as Home secretary ...,2022-10-25,12061,2073,50089,604,1271722,123,Ongoing News Story,Ongoing News Stories like 'Brexit',1484601166080081920,Russo-Ukrainian conflict,NaN,1505285549820682245,NaN,NaN,64827
726789,Disgusted to hear Suella Braverman say there i...,2022-10-31,9959,5332,48714,601,1138495,131,Unified Twitter Taxonomy,A taxonomy of user interests.,857212714300489729,Liz Truss,Liz Truss,1504117989150543874,NaN,NaN,64606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313476,Every time Liz Truss talks about her mandate o...,2022-09-05,0,1,5,0,496886,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1498261788160364544,NaN,NaN,6
92547,She is left-handed [URL],2022-02-22,2,2,2,0,137194,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1492089230134239234,NaN,NaN,6
425549,"@itaisher I think I just mean the ""de facto"" t...",2022-10-25,0,2,4,0,698875,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1497706081388269571,NaN,NaN,6
73778,@Elemjay1 @trussliz @one_indy_clan Well said L...,2022-02-24,0,0,6,0,110629,131,Unified Twitter Taxonomy,A taxonomy of user interests.,847878884917886977,Politics,Politics,1492280785415585792,NaN,NaN,6


In [319]:
df = df.sort_values(by=['public_metrics', 'hashtags'], ascending=False)
df

,text,created_at,retweet_count,reply_count,like_count,quote_count,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,reference_tweet_id,hashtags,cashtags,public_metrics
345107,Yes @trussliz absolutely smashed it babe!!!,2022-09-05,7747,2611,145129,675,557681,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1196447117297905665,News outlets,NaN,1497534323616096260,NaN,NaN,156162
822052,The rise of Suella Braverman is one of the gen...,2022-10-25,9759,1373,56143,473,1274643,131,Unified Twitter Taxonomy,A taxonomy of user interests.,847878884917886977,Politics,Politics,1505193990773260289,NaN,NaN,67748
277038,"General Election please! 81,000 Tory members v...",2022-09-06,5299,5231,54180,661,419155,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1498344952790671360,NaN,NaN,65371
819760,Appointing Suella Braverman as Home secretary ...,2022-10-25,12061,2073,50089,604,1271722,123,Ongoing News Story,Ongoing News Stories like 'Brexit',1484601166080081920,Russo-Ukrainian conflict,NaN,1505285549820682245,NaN,NaN,64827
726789,Disgusted to hear Suella Braverman say there i...,2022-10-31,9959,5332,48714,601,1138495,131,Unified Twitter Taxonomy,A taxonomy of user interests.,857212714300489729,Liz Truss,Liz Truss,1504117989150543874,NaN,NaN,64606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313476,Every time Liz Truss talks about her mandate o...,2022-09-05,0,1,5,0,496886,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1498261788160364544,NaN,NaN,6
92547,She is left-handed [URL],2022-02-22,2,2,2,0,137194,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1492089230134239234,NaN,NaN,6
425549,"@itaisher I think I just mean the ""de facto"" t...",2022-10-25,0,2,4,0,698875,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1497706081388269571,NaN,NaN,6
73778,@Elemjay1 @trussliz @one_indy_clan Well said L...,2022-02-24,0,0,6,0,110629,131,Unified Twitter Taxonomy,A taxonomy of user interests.,847878884917886977,Politics,Politics,1492280785415585792,NaN,NaN,6


In [322]:
df['text'].str.split().apply(len).value_counts()

text
12     3621
13     3561
14     3536
15     3458
17     3404
       ... 
103       1
107       1
102       1
78        1
1         1
Name: count, Length: 104, dtype: int64

In [323]:
df[df['text'].str.split().apply(len) < 10] # can eliminate about 10% of the data

,text,created_at,retweet_count,reply_count,like_count,quote_count,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,reference_tweet_id,hashtags,cashtags,public_metrics
345107,Yes @trussliz absolutely smashed it babe!!!,2022-09-05,7747,2611,145129,675,557681,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1196447117297905665,News outlets,NaN,1497534323616096260,NaN,NaN,156162
709378,Congratulations on the new job @SuellaBraverma...,2022-09-06,4343,2975,25830,863,1112570,10,Person,Named people in the world like Nelson Mandela,1070735170720808961,Jacob Rees-Mogg,British politician,1506310549994446859,NaN,NaN,34011
876164,Unless they try to come here... [URL],2022-02-24,2029,203,21517,56,1362564,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1070032753834438656,Political figures,Politician,1504280357956464646,NaN,NaN,23805
821406,Suella Braverman back in Home office. Heaven h...,2022-10-25,2675,741,17703,182,1273851,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1070735181206573056,Bill Cash,British politician,1505120309367783426,NaN,NaN,21301
315292,It is. Behind. A paywall. [URL],2022-09-05,1756,270,19034,120,501814,167,Animals,", for animals and types of animals, e.g., cats...",854025872537403392,Poodles,Poodles,1498273065867923459,NaN,NaN,21180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669342,@CarolineLucas @DominicRaab Who funds you?,2022-05-08,1,0,5,0,1051324,131,Unified Twitter Taxonomy,A taxonomy of user interests.,1070032753834438656,Political figures,Politician,1495867739201757186,NaN,NaN,6
425382,@GregHands @trussliz Really????? [URL],2022-10-25,1,0,5,0,698660,35,Politician,"Politicians in the world, like Joe Biden",857212714300489729,Liz Truss,Liz Truss,1497665557063208963,NaN,NaN,6
923010,@WillBlackWriter @gnomeicide @pritipatel @Bori...,2022-04-19,2,1,3,0,1426072,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1504090119048945671,NaN,NaN,6
92547,She is left-handed [URL],2022-02-22,2,2,2,0,137194,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1492089230134239234,NaN,NaN,6


In [324]:
df = df[df['text'].str.split().apply(len) >= 10] ## only keeping the ones which would be useful

In [325]:
df = df.reset_index(drop=True)
df

,text,created_at,retweet_count,reply_count,like_count,quote_count,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,reference_tweet_id,hashtags,cashtags,public_metrics
0,The rise of Suella Braverman is one of the gen...,2022-10-25,9759,1373,56143,473,1274643,131,Unified Twitter Taxonomy,A taxonomy of user interests.,847878884917886977,Politics,Politics,1505193990773260289,NaN,NaN,67748
1,"General Election please! 81,000 Tory members v...",2022-09-06,5299,5231,54180,661,419155,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1498344952790671360,NaN,NaN,65371
2,Appointing Suella Braverman as Home secretary ...,2022-10-25,12061,2073,50089,604,1271722,123,Ongoing News Story,Ongoing News Stories like 'Brexit',1484601166080081920,Russo-Ukrainian conflict,NaN,1505285549820682245,NaN,NaN,64827
3,Disgusted to hear Suella Braverman say there i...,2022-10-31,9959,5332,48714,601,1138495,131,Unified Twitter Taxonomy,A taxonomy of user interests.,857212714300489729,Liz Truss,Liz Truss,1504117989150543874,NaN,NaN,64606
4,Just bumped into Suella Braverman who said to ...,2022-10-24,6291,3227,49314,540,1276819,35,Politician,"Politicians in the world, like Joe Biden",1070735213792129024,Brandon Lewis,British politician,1505216054406037505,NaN,NaN,59372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115186,"@MichelleDewbs @trussliz She was clear, concis...",2022-09-07,0,0,6,0,288965,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1491787735610191872,NaN,NaN,6
115187,@SamCoatesSky @RupaHuq @SuellaBraverman Extrao...,2022-02-10,0,1,5,0,1084156,35,Politician,"Politicians in the world, like Joe Biden",857212714300489729,Liz Truss,Liz Truss,1496096138180673541,NaN,NaN,6
115188,Every time Liz Truss talks about her mandate o...,2022-09-05,0,1,5,0,496886,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1498261788160364544,NaN,NaN,6
115189,"@itaisher I think I just mean the ""de facto"" t...",2022-10-25,0,2,4,0,698875,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1497706081388269571,NaN,NaN,6


In [326]:
df['text'].iloc[0]

'The rise of Suella Braverman is one of the genuine mysteries of modern politics. She has no discernible charm, intellect or talent and projects cruelty like toxic radiation. Shame on any leader who puts her within a mile of power.'

0          6
1         40
2         22
3         39
4         44
          ..
127709    14
127710    57
127711    51
127712    30
127713    28
Name: text, Length: 127714, dtype: int64

In [327]:
df['uniqueID'] = df.index
df['uniqueID'] = 'antisexm24_uk-' + df['uniqueID'].astype(str)
df.insert(0, 'uniqueID', df.pop('uniqueID'))  ## repositioning it to the front
df

,uniqueID,text,created_at,retweet_count,reply_count,like_count,quote_count,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,reference_tweet_id,hashtags,cashtags,public_metrics
0,antisexm24_uk-0,The rise of Suella Braverman is one of the gen...,2022-10-25,9759,1373,56143,473,1274643,131,Unified Twitter Taxonomy,A taxonomy of user interests.,847878884917886977,Politics,Politics,1505193990773260289,NaN,NaN,67748
1,antisexm24_uk-1,"General Election please! 81,000 Tory members v...",2022-09-06,5299,5231,54180,661,419155,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1498344952790671360,NaN,NaN,65371
2,antisexm24_uk-2,Appointing Suella Braverman as Home secretary ...,2022-10-25,12061,2073,50089,604,1271722,123,Ongoing News Story,Ongoing News Stories like 'Brexit',1484601166080081920,Russo-Ukrainian conflict,NaN,1505285549820682245,NaN,NaN,64827
3,antisexm24_uk-3,Disgusted to hear Suella Braverman say there i...,2022-10-31,9959,5332,48714,601,1138495,131,Unified Twitter Taxonomy,A taxonomy of user interests.,857212714300489729,Liz Truss,Liz Truss,1504117989150543874,NaN,NaN,64606
4,antisexm24_uk-4,Just bumped into Suella Braverman who said to ...,2022-10-24,6291,3227,49314,540,1276819,35,Politician,"Politicians in the world, like Joe Biden",1070735213792129024,Brandon Lewis,British politician,1505216054406037505,NaN,NaN,59372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115186,antisexm24_uk-115186,"@MichelleDewbs @trussliz She was clear, concis...",2022-09-07,0,0,6,0,288965,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1491787735610191872,NaN,NaN,6
115187,antisexm24_uk-115187,@SamCoatesSky @RupaHuq @SuellaBraverman Extrao...,2022-02-10,0,1,5,0,1084156,35,Politician,"Politicians in the world, like Joe Biden",857212714300489729,Liz Truss,Liz Truss,1496096138180673541,NaN,NaN,6
115188,antisexm24_uk-115188,Every time Liz Truss talks about her mandate o...,2022-09-05,0,1,5,0,496886,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1498261788160364544,NaN,NaN,6
115189,antisexm24_uk-115189,"@itaisher I think I just mean the ""de facto"" t...",2022-10-25,0,2,4,0,698875,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1497706081388269571,NaN,NaN,6


In [328]:
#df.to_csv('data/orig/original_full_dataset.csv', index=False)

In [329]:
df.columns

Index(['uniqueID', 'text', 'created_at', 'retweet_count', 'reply_count',
       'like_count', 'quote_count', 'index', 'domain.id', 'domain.name',
       'domain.description', 'entity.id', 'entity.name', 'entity.description',
       'reference_tweet_id', 'hashtags', 'cashtags', 'public_metrics'],
      dtype='object')

In [330]:
df_metadata = df[['uniqueID', 'created_at', 'retweet_count', 'reply_count',
       'like_count', 'quote_count', 'index', 'domain.id', 'domain.name',
       'domain.description', 'entity.id', 'entity.name', 'entity.description',
       'reference_tweet_id', 'hashtags', 'cashtags', 'public_metrics']]
#df_metadata.to_csv('data/orig/metadata_full_dataset.csv', index=False)

In [333]:
df_texts = df[['uniqueID', 'text']]
df_texts

,uniqueID,text
0,antisexm24_uk-0,The rise of Suella Braverman is one of the gen...
1,antisexm24_uk-1,"General Election please! 81,000 Tory members v..."
2,antisexm24_uk-2,Appointing Suella Braverman as Home secretary ...
3,antisexm24_uk-3,Disgusted to hear Suella Braverman say there i...
4,antisexm24_uk-4,Just bumped into Suella Braverman who said to ...
...,...,...
115186,antisexm24_uk-115186,"@MichelleDewbs @trussliz She was clear, concis..."
115187,antisexm24_uk-115187,@SamCoatesSky @RupaHuq @SuellaBraverman Extrao...
115188,antisexm24_uk-115188,Every time Liz Truss talks about her mandate o...
115189,antisexm24_uk-115189,"@itaisher I think I just mean the ""de facto"" t..."


In [334]:
df_pilot = pd.read_csv('data/pilot/Annotation_Chico 1.csv', encoding='unicode_escape')

df_taken = pd.read_csv('data/pilot/Annotations_Aditi 1.csv', encoding='unicode_escape')
df_pilot = pd.concat([df_pilot, df_taken], axis = 0)

df_taken = pd.read_csv('data/pilot/Annotations_Aditi 2.csv', encoding='unicode_escape')
df_pilot = pd.concat([df_pilot, df_taken], axis = 0)

df_taken = pd.read_csv('data/pilot/Annotations_Aditi 3.csv', encoding='unicode_escape')
df_pilot = pd.concat([df_pilot, df_taken], axis = 0)

df_taken = pd.read_csv('data/pilot/Annotations_Aditi 4.csv', encoding='unicode_escape')
df_pilot = pd.concat([df_pilot, df_taken], axis = 0)

df_taken = pd.read_csv('data/pilot/Annotations_Susan 1.csv', encoding='unicode_escape')
df_pilot = pd.concat([df_pilot, df_taken], axis = 0)

df_taken = pd.read_csv('data/pilot/Annotations_Susan 2.csv', encoding='unicode_escape')
df_pilot = pd.concat([df_pilot, df_taken], axis = 0)

df_taken = pd.read_csv('data/pilot/Annotations_Susan 3.csv', encoding='unicode_escape')
df_pilot = pd.concat([df_pilot, df_taken], axis = 0)

df_pilot = df_pilot.reset_index().rename(columns={'level_0': 'index'})

In [335]:
df_pilot

,index,Unnamed: 0,text,label,index,id,type,created_at,Unnamed: 0.1,Unnamed: 7
0,0,100,@zarahsultana @pritipatel You??ll never be mi...,neither,NaN,NaN,NaN,NaN,NaN,NaN
1,1,101,@diane25026692 @10DowningStreet @SuellaBraverm...,neither,NaN,NaN,NaN,NaN,NaN,NaN
2,2,102,@zarahsultana @pritipatel Cruel because she wa...,neither,NaN,NaN,NaN,NaN,NaN,NaN
3,3,103,@Conservatives @theresecoffey And we thought R...,neither,NaN,NaN,NaN,NaN,NaN,NaN
4,4,104,@_HenryBolton @trussliz @SuellaBraverman Blath...,neither,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
895,95,399,Boris Johnson visited Gigg Lane stadium today....,neither,1426.0,1.518660e+18,replied_to,25/04/2022,NaN,NaN
896,96,831,@EmilyThornberry @PETAUK This at labour wonâ...,neither,2896.0,1.518630e+18,NaN,25/04/2022,NaN,NaN
897,97,49,Queen ð @AngelaRayner https://t.co/7HDiEHy3sJ,neither,189.0,1.518720e+18,NaN,25/04/2022,NaN,NaN
898,98,567,UK Prime Minister Boris Johnson lashed out at ...,neither,2004.0,1.518640e+18,replied_to,25/04/2022,NaN,NaN


In [336]:
df_pilot.columns

Index(['index', 'Unnamed: 0', 'text', 'label', 'index', 'id', 'type',
       'created_at', 'Unnamed: 0.1', 'Unnamed: 7'],
      dtype='object')

In [337]:
df_pilot = df_pilot[['index', 'text', 'label', 'index', 'id', 'type', 'created_at']]
#df_pilot.to_csv('data/pilot/combined_data.csv')

In [338]:
df_pilot['text'] = df_pilot['text'].apply(lambda x: usual_preprocess(x))

/tmp/ipykernel_768895/1436801552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pilot['text'] = df_pilot['text'].apply(lambda x: usual_preprocess(x))


In [339]:
df_pilot['label'].value_counts()

label
neither        787
anti-sexism     58
sexism          55
Name: count, dtype: int64

In [340]:
# check the IDs of the repeated text
list_ids = df_texts.loc[df_texts['text'].isin(df_pilot['text']) == True]['uniqueID'].to_list()
list_ids

['antisexm24_uk-767',
 'antisexm24_uk-4902',
 'antisexm24_uk-7085',
 'antisexm24_uk-8674',
 'antisexm24_uk-16369',
 'antisexm24_uk-17619',
 'antisexm24_uk-18904',
 'antisexm24_uk-19203',
 'antisexm24_uk-23680',
 'antisexm24_uk-24072',
 'antisexm24_uk-24980',
 'antisexm24_uk-25289',
 'antisexm24_uk-26883',
 'antisexm24_uk-31357',
 'antisexm24_uk-33667',
 'antisexm24_uk-36025',
 'antisexm24_uk-42918',
 'antisexm24_uk-43014',
 'antisexm24_uk-46621',
 'antisexm24_uk-47926',
 'antisexm24_uk-50421',
 'antisexm24_uk-51213',
 'antisexm24_uk-57605',
 'antisexm24_uk-60097',
 'antisexm24_uk-63838',
 'antisexm24_uk-65724',
 'antisexm24_uk-67123',
 'antisexm24_uk-69984',
 'antisexm24_uk-71797',
 'antisexm24_uk-72705',
 'antisexm24_uk-72920',
 'antisexm24_uk-73288',
 'antisexm24_uk-75117',
 'antisexm24_uk-78580',
 'antisexm24_uk-79263',
 'antisexm24_uk-81584',
 'antisexm24_uk-82534',
 'antisexm24_uk-82544',
 'antisexm24_uk-85512',
 'antisexm24_uk-90666',
 'antisexm24_uk-90843',
 'antisexm24_uk-91882

In [341]:
df_pilot.loc[df_pilot['text'].isin(df_texts['text']) == True].index.to_list()

[15,
 16,
 41,
 95,
 100,
 112,
 167,
 178,
 183,
 190,
 208,
 235,
 289,
 318,
 349,
 355,
 361,
 363,
 383,
 390,
 419,
 520,
 528,
 537,
 540,
 541,
 542,
 566,
 577,
 585,
 620,
 664,
 669,
 671,
 679,
 698,
 718,
 730,
 768,
 779,
 821,
 861,
 880,
 883]

In [342]:
df_labeled = df_texts[df_texts['uniqueID'].isin(list_ids)]
df_labeled = pd.merge(df_labeled, df_pilot[['text', 'label']], on = 'text')
df_labeled

,uniqueID,text,label
0,antisexm24_uk-767,"The Durham ""beergate"" affair is becoming more ...",neither
1,antisexm24_uk-4902,"""She calls them utter scum, which is the filth...",neither
2,antisexm24_uk-7085,They will not stand up. Until whichever malici...,neither
3,antisexm24_uk-8674,Suella Braverman has entered Number 10. It is ...,neither
4,antisexm24_uk-16369,"""Somebody sent me the screenshot of it on Twit...",neither
5,antisexm24_uk-17619,"I totally get the Angela Rayner thing, I mean ...",sexism
6,antisexm24_uk-18904,@errjustsaying @LadyRebecca_1 I am absolutely ...,neither
7,antisexm24_uk-19203,Bit weird how tories boast of having given us ...,anti-sexism
8,antisexm24_uk-23680,@AngelaRayner @LiseMcNally @GranadaReports May...,sexism
9,antisexm24_uk-24072,@AliBarry25 @jessphillips Good point. No menti...,neither


In [343]:
#df_labeled.to_csv('data/annotations/labeled_data_1.csv')

In [344]:
df_selected = df[~df['uniqueID'].isin(list_ids)]
df_selected

,uniqueID,text,created_at,retweet_count,reply_count,like_count,quote_count,index,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description,reference_tweet_id,hashtags,cashtags,public_metrics
0,antisexm24_uk-0,The rise of Suella Braverman is one of the gen...,2022-10-25,9759,1373,56143,473,1274643,131,Unified Twitter Taxonomy,A taxonomy of user interests.,847878884917886977,Politics,Politics,1505193990773260289,NaN,NaN,67748
1,antisexm24_uk-1,"General Election please! 81,000 Tory members v...",2022-09-06,5299,5231,54180,661,419155,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1498344952790671360,NaN,NaN,65371
2,antisexm24_uk-2,Appointing Suella Braverman as Home secretary ...,2022-10-25,12061,2073,50089,604,1271722,123,Ongoing News Story,Ongoing News Stories like 'Brexit',1484601166080081920,Russo-Ukrainian conflict,NaN,1505285549820682245,NaN,NaN,64827
3,antisexm24_uk-3,Disgusted to hear Suella Braverman say there i...,2022-10-31,9959,5332,48714,601,1138495,131,Unified Twitter Taxonomy,A taxonomy of user interests.,857212714300489729,Liz Truss,Liz Truss,1504117989150543874,NaN,NaN,64606
4,antisexm24_uk-4,Just bumped into Suella Braverman who said to ...,2022-10-24,6291,3227,49314,540,1276819,35,Politician,"Politicians in the world, like Joe Biden",1070735213792129024,Brandon Lewis,British politician,1505216054406037505,NaN,NaN,59372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115186,antisexm24_uk-115186,"@MichelleDewbs @trussliz She was clear, concis...",2022-09-07,0,0,6,0,288965,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1491787735610191872,NaN,NaN,6
115187,antisexm24_uk-115187,@SamCoatesSky @RupaHuq @SuellaBraverman Extrao...,2022-02-10,0,1,5,0,1084156,35,Politician,"Politicians in the world, like Joe Biden",857212714300489729,Liz Truss,Liz Truss,1496096138180673541,NaN,NaN,6
115188,antisexm24_uk-115188,Every time Liz Truss talks about her mandate o...,2022-09-05,0,1,5,0,496886,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1498261788160364544,NaN,NaN,6
115189,antisexm24_uk-115189,"@itaisher I think I just mean the ""de facto"" t...",2022-10-25,0,2,4,0,698875,10,Person,Named people in the world like Nelson Mandela,857212714300489729,Liz Truss,Liz Truss,1497706081388269571,NaN,NaN,6


In [ ]:
### 

### Make the changes from here on

In [136]:
annotator_list = ['Annotator 1', 'Annotator 2', 'Annotator 3', 'Annotator 4', 'Annotator 5', 'Annotator 6', 'Annotator 7']

In [346]:
df_selected_test = df_selected[:238]
df_selected_test = df_selected_test[['uniqueID', 'text']]
df_selected_test


,uniqueID,text
0,antisexm24_uk-0,The rise of Suella Braverman is one of the gen...
1,antisexm24_uk-1,"General Election please! 81,000 Tory members v..."
2,antisexm24_uk-2,Appointing Suella Braverman as Home secretary ...
3,antisexm24_uk-3,Disgusted to hear Suella Braverman say there i...
4,antisexm24_uk-4,Just bumped into Suella Braverman who said to ...
...,...,...
233,antisexm24_uk-233,He stood by Boris Johnson till the bitter end ...
234,antisexm24_uk-234,"Rishi Sunak""s speaking style makes Liz Truss l..."
235,antisexm24_uk-235,"""Multiple breaches"" by Suella Braverman accord..."
236,antisexm24_uk-236,@pritipatel @BorisJohnson It has been the emba...


In [243]:
#df_selected_test.to_excel('selected_list.xlsx')

In [263]:
#df_selected_test["choice 1"] = np.random.choice(annotator_list, size=len(df_selected_test))
df_selected_test["choice1"] = np.random.choice(annotator_list, p = (0.15, 0.14, 0.15, 0.14, 0.14, 0.14, 0.14), size=len(df_selected_test)) 
df_selected_test['choice2'] = df_selected_test.apply(lambda x: np.random.choice(list(set(annotator_list) - {x.choice1})), axis=1)
df_selected_test['choice3'] = df_selected_test.apply(lambda x: np.random.choice(list(set(annotator_list) - {x.choice1} - {x.choice2})), axis =1)
df_selected_test

,uniqueID,text,choice1,choice2,choice3
0,antisexm24_uk-0,The rise of Suella Braverman is one of the gen...,Annotator 2,Annotator 3,Annotator 5
1,antisexm24_uk-1,"General Election please! bl,ooo Tory members v...",Annotator 7,Annotator 4,Annotator 5
2,antisexm24_uk-2,Appointing Suella Braverman as Home secretary ...,Annotator 1,Annotator 3,Annotator 4
3,antisexm24_uk-3,Disgusted to hear Suella Braverman say there i...,Annotator 6,Annotator 5,Annotator 7
4,antisexm24_uk-4,Just bumped into Suella Braverman who said to ...,Annotator 1,Annotator 3,Annotator 6
...,...,...,...,...,...
233,antisexm24_uk-233,He stood by Boris Johnson till the bitter end ...,Annotator 5,Annotator 3,Annotator 2
234,antisexm24_uk-234,"Rishi Sunak""s speaking style makes Liz Truss l...",Annotator 1,Annotator 4,Annotator 3
235,antisexm24_uk-235,"""Multiple breaches"" by Suella Braverman accord...",Annotator 5,Annotator 2,Annotator 7
236,antisexm24_uk-236,@pritipatel @BorisJohnson It has been the emba...,Annotator 7,Annotator 1,Annotator 5


In [264]:
df_selected_test.apply(lambda x: x.choice1 == x.choice2 == x.choice3, axis=1).value_counts() # none of them repeat across rows

False    238
Name: count, dtype: int64

In [265]:
df_selected_test.choice1.value_counts(), df_selected_test.choice2.value_counts(), df_selected_test.choice3.value_counts()

(choice1
 Annotator 5    46
 Annotator 1    37
 Annotator 4    36
 Annotator 7    35
 Annotator 3    34
 Annotator 2    25
 Annotator 6    25
 Name: count, dtype: int64,
 choice2
 Annotator 2    41
 Annotator 3    40
 Annotator 4    37
 Annotator 7    37
 Annotator 5    28
 Annotator 6    28
 Annotator 1    27
 Name: count, dtype: int64,
 choice3
 Annotator 7    37
 Annotator 1    37
 Annotator 5    35
 Annotator 6    35
 Annotator 3    33
 Annotator 2    32
 Annotator 4    29
 Name: count, dtype: int64)

In [266]:
df_selected_test

,uniqueID,text,choice1,choice2,choice3
0,antisexm24_uk-0,The rise of Suella Braverman is one of the gen...,Annotator 2,Annotator 3,Annotator 5
1,antisexm24_uk-1,"General Election please! bl,ooo Tory members v...",Annotator 7,Annotator 4,Annotator 5
2,antisexm24_uk-2,Appointing Suella Braverman as Home secretary ...,Annotator 1,Annotator 3,Annotator 4
3,antisexm24_uk-3,Disgusted to hear Suella Braverman say there i...,Annotator 6,Annotator 5,Annotator 7
4,antisexm24_uk-4,Just bumped into Suella Braverman who said to ...,Annotator 1,Annotator 3,Annotator 6
...,...,...,...,...,...
233,antisexm24_uk-233,He stood by Boris Johnson till the bitter end ...,Annotator 5,Annotator 3,Annotator 2
234,antisexm24_uk-234,"Rishi Sunak""s speaking style makes Liz Truss l...",Annotator 1,Annotator 4,Annotator 3
235,antisexm24_uk-235,"""Multiple breaches"" by Suella Braverman accord...",Annotator 5,Annotator 2,Annotator 7
236,antisexm24_uk-236,@pritipatel @BorisJohnson It has been the emba...,Annotator 7,Annotator 1,Annotator 5


In [268]:
#df_selected_test.to_csv('selected_test.csv')

In [ ]:
'''df_selected_test['choice 2'] = 
while (self_review := df_selected_test['choice 2'] == df_selected_test['choice 1']).any():
    reviewers = np.random.choice(users, self_review.sum())
    df_selected_test.loc[self_review, 'reviewer_id'] = reviewers'''

In [356]:
df_annot = pd.read_csv('data/selected_test_1st_1-238.csv')
df_annot

,Unnamed: 0,uniqueID,text,choice1,choice2,choice3
0,0,antisexm24_uk-0,The rise of Suella Braverman is one of the gen...,Annotator 2,Annotator 3,Annotator 5
1,1,antisexm24_uk-1,"General Election please! 81,000 Tory members v...",Annotator 7,Annotator 6,Annotator 5
2,2,antisexm24_uk-2,Appointing Suella Braverman as Home secretary ...,Annotator 1,Annotator 3,Annotator 4
3,3,antisexm24_uk-3,Disgusted to hear Suella Braverman say there i...,Annotator 6,Annotator 5,Annotator 7
4,4,antisexm24_uk-4,Just bumped into Suella Braverman who said to ...,Annotator 1,Annotator 3,Annotator 6
...,...,...,...,...,...,...
233,233,antisexm24_uk-233,He stood by Boris Johnson till the bitter end ...,Annotator 5,Annotator 3,Annotator 2
234,234,antisexm24_uk-234,"Rishi Sunak""s speaking style makes Liz Truss l...",Annotator 1,Annotator 4,Annotator 3
235,235,antisexm24_uk-235,"""Multiple breaches"" by Suella Braverman accord...",Annotator 5,Annotator 2,Annotator 7
236,236,antisexm24_uk-236,@pritipatel @BorisJohnson It has been the emba...,Annotator 7,Annotator 1,Annotator 5


In [353]:
'''df_annot = df_annot.drop(columns='text')
df_annot = pd.merge(df_annot, df_selected_test, on='uniqueID')
col = df_annot.pop('text')
df_annot.insert(2, col.name, col)
'''
df_annot

,Unnamed: 0,uniqueID,text,choice1,choice2,choice3
0,0,antisexm24_uk-0,The rise of Suella Braverman is one of the gen...,Annotator 2,Annotator 3,Annotator 5
1,1,antisexm24_uk-1,"General Election please! 81,000 Tory members v...",Annotator 7,Annotator 6,Annotator 5
2,2,antisexm24_uk-2,Appointing Suella Braverman as Home secretary ...,Annotator 1,Annotator 3,Annotator 4
3,3,antisexm24_uk-3,Disgusted to hear Suella Braverman say there i...,Annotator 6,Annotator 5,Annotator 7
4,4,antisexm24_uk-4,Just bumped into Suella Braverman who said to ...,Annotator 1,Annotator 3,Annotator 6
...,...,...,...,...,...,...
233,233,antisexm24_uk-233,He stood by Boris Johnson till the bitter end ...,Annotator 5,Annotator 3,Annotator 2
234,234,antisexm24_uk-234,"Rishi Sunak""s speaking style makes Liz Truss l...",Annotator 1,Annotator 4,Annotator 3
235,235,antisexm24_uk-235,"""Multiple breaches"" by Suella Braverman accord...",Annotator 5,Annotator 2,Annotator 7
236,236,antisexm24_uk-236,@pritipatel @BorisJohnson It has been the emba...,Annotator 7,Annotator 1,Annotator 5


In [357]:
df_annot.apply(lambda x: x.choice1 == x.choice2 == x.choice3, axis=1).value_counts()

False    238
Name: count, dtype: int64

In [358]:
df_annot.choice1.value_counts(), df_annot.choice2.value_counts(), df_annot.choice3.value_counts()

(choice1
 Annotator 1    43
 Annotator 5    37
 Annotator 4    36
 Annotator 7    35
 Annotator 3    34
 Annotator 2    27
 Annotator 6    26
 Name: count, dtype: int64,
 choice2
 Annotator 2    41
 Annotator 3    40
 Annotator 6    39
 Annotator 4    35
 Annotator 5    28
 Annotator 7    28
 Annotator 1    27
 Name: count, dtype: int64,
 choice3
 Annotator 7    37
 Annotator 1    37
 Annotator 5    35
 Annotator 6    35
 Annotator 3    33
 Annotator 2    32
 Annotator 4    29
 Name: count, dtype: int64)

In [359]:
df_annot_1 = df_annot[(df_annot['choice1'] == 'Annotator 1') | (df_annot['choice2'] == 'Annotator 1') | (df_annot['choice3'] == 'Annotator 1')][['uniqueID', 'text']]
df_annot_1['label'] = ''

df_annot_2 = df_annot[(df_annot['choice1'] == 'Annotator 2') | (df_annot['choice2'] == 'Annotator 2') | (df_annot['choice3'] == 'Annotator 2')][['uniqueID', 'text']]
df_annot_2['label'] = ''

df_annot_3 = df_annot[(df_annot['choice1'] == 'Annotator 3') | (df_annot['choice2'] == 'Annotator 3') | (df_annot['choice3'] == 'Annotator 3')][['uniqueID', 'text']]
df_annot_3['label'] = ''

df_annot_4 = df_annot[(df_annot['choice1'] == 'Annotator 4') | (df_annot['choice2'] == 'Annotator 4') | (df_annot['choice3'] == 'Annotator 4')][['uniqueID', 'text']]
df_annot_4['label'] = ''

df_annot_5 = df_annot[(df_annot['choice1'] == 'Annotator 5') | (df_annot['choice2'] == 'Annotator 5') | (df_annot['choice3'] == 'Annotator 5')][['uniqueID', 'text']]
df_annot_5['label'] = ''

df_annot_6 = df_annot[(df_annot['choice1'] == 'Annotator 6') | (df_annot['choice2'] == 'Annotator 6') | (df_annot['choice3'] == 'Annotator 6')][['uniqueID', 'text']]
df_annot_6['label'] = ''

df_annot_7 = df_annot[(df_annot['choice1'] == 'Annotator 7') | (df_annot['choice2'] == 'Annotator 7') | (df_annot['choice3'] == 'Annotator 7')][['uniqueID', 'text']]
df_annot_7['label'] = ''

df_annot_7

,uniqueID,text,label
1,antisexm24_uk-1,"General Election please! 81,000 Tory members v...",
3,antisexm24_uk-3,Disgusted to hear Suella Braverman say there i...,
6,antisexm24_uk-6,I have worked with 8 Home Secretaries and I ca...,
7,antisexm24_uk-7,I would like to congratulate our next Prime Mi...,
9,antisexm24_uk-9,Have I got this right? Liz Truss is intending ...,
...,...,...,...
230,antisexm24_uk-230,Full Rent PAID! All energy bills PAID! 5 YEAR ...,
231,antisexm24_uk-231,This Morning: @BorisJohnson leaves Downing St ...,
235,antisexm24_uk-235,"""Multiple breaches"" by Suella Braverman accord...",
236,antisexm24_uk-236,@pritipatel @BorisJohnson It has been the emba...,


In [361]:
'''df_annot_1.to_csv('1st_batch_annot_1.csv')
df_annot_2.to_csv('1st_batch_annot_2.csv')
df_annot_3.to_csv('1st_batch_annot_3.csv')
df_annot_4.to_csv('1st_batch_annot_4.csv')
df_annot_5.to_csv('1st_batch_annot_5.csv')
df_annot_6.to_csv('1st_batch_annot_6.csv')
df_annot_7.to_csv('1st_batch_annot_7.csv')'''